# Omdena  - Milan Chapter Agrifoods
## AI for Sustainable agri-food systems: use of Satellite Imagery
### Exploratory analysis of cereals production in Italy 2006-2022
#### Author: Maria Fisher 


The main objective of this study is to have gather information about crop production in Italy for the period of 2006-2022. 

Crop dataset used in this study was downloaded from the Italian National Institute of Statistics (Istat).



In [1]:
import warnings 
warnings.filterwarnings("ignore")

import os
import pandas as pd
pd.options.display.float_format = "{:.2f}".format
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns 
import scipy 
import sklearn
import geopandas as gpd
import pgeocode
import folium
import sys
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot

In [8]:
cereals = pd.read_csv('Italy_crop_data/cereals_legumes.csv',skipinitialspace=True)
cereals.head()

,ITTER107,Territory,TIPO_DATO5,Data type,AGRI_MADRE,Type of crop,TIME,Select time,Value,Flag Codes,Flags
0,ITC11,Torino,ART,total area - hectares,COMMEAT,common wheat,2006,2006,17100.00,NaN,NaN
1,ITC11,Torino,ART,total area - hectares,COMMEAT,common wheat,2007,2007,21502.00,NaN,NaN
2,ITC11,Torino,ART,total area - hectares,COMMEAT,common wheat,2008,2008,23400.00,NaN,NaN
3,ITC11,Torino,ART,total area - hectares,COMMEAT,common wheat,2009,2009,23167.00,e,estimate data
4,ITC11,Torino,ART,total area - hectares,COMMEAT,common wheat,2010,2010,20904.00,e,estimate data


## Pre-processing dataset 

In [9]:
# Drop Columns
cereals = cereals.drop(columns =['ITTER107','TIPO_DATO5','AGRI_MADRE', 'TIME', 'Flag Codes','Flags' ])
cereals

,Territory,Data type,Type of crop,Select time,Value
0,Torino,total area - hectares,common wheat,2006,17100.00
1,Torino,total area - hectares,common wheat,2007,21502.00
2,Torino,total area - hectares,common wheat,2008,23400.00
3,Torino,total area - hectares,common wheat,2009,23167.00
4,Torino,total area - hectares,common wheat,2010,20904.00
...,...,...,...,...,...
66260,Pesaro e Urbino,total area - hectares,rye and winter cereal mixtures (maslin),2022,3.00
66261,Roma,total area - hectares,rye and winter cereal mixtures (maslin),2022,20.00
66262,Valle d'Aosta / Vallée d'Aoste,total area - hectares,winter barley,2022,1.00
66263,Torino,total area - hectares,spring barley,2022,220.00


In [10]:
# Rename Columns
cereals = cereals.rename(columns = {'Select time':'Year', 'Type of crop':'Type_crop', 'Data type':'Data_type', 'Territory':'City'})


In [11]:
cereals = cereals[cereals["Year"] < 2022]

In [12]:
def show_info(cereals):
    print('DATASET SHAPE: ', cereals.shape, '\n')
    print('-'*50)
    print('FEATURE DATA TYPES:')
    print(cereals.info())
    print('\n', '-'*50)
    print('NUMBER OF UNIQUE VALUES PER FEATURE:', '\n')
    print(cereals.nunique())
    print('\n', '-'*50)
    print('NULL VALUES PER FEATURE')
    print(cereals.isnull().sum())
show_info(cereals)

DATASET SHAPE:  (61321, 5) 

--------------------------------------------------
FEATURE DATA TYPES:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 61321 entries, 0 to 66252
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   City       61321 non-null  object 
 1   Data_type  61321 non-null  object 
 2   Type_crop  61321 non-null  object 
 3   Year       61321 non-null  int64  
 4   Value      61321 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 2.8+ MB
None

 --------------------------------------------------
NUMBER OF UNIQUE VALUES PER FEATURE: 

City           112
Data_type        3
Type_crop       29
Year            16
Value        13582
dtype: int64

 --------------------------------------------------
NULL VALUES PER FEATURE
City         0
Data_type    0
Type_crop    0
Year         0
Value        0
dtype: int64


## Cities in Italy producing cereals

In [13]:
# Check cities names
cereals.City.unique()


array(['Torino', 'Vercelli', 'Biella', 'Verbano-Cusio-Ossola', 'Novara',
       'Cuneo', 'Asti', 'Alessandria', "Valle d'Aosta / Vallée d'Aoste",
       'Savona', 'Genova', 'La Spezia', 'Varese', 'Como', 'Lecco',
       'Sondrio', 'Milano', 'Bergamo', 'Brescia', 'Pavia', 'Lodi',
       'Cremona', 'Mantova', 'Trentino Alto Adige / Südtirol', 'Verona',
       'Vicenza', 'Belluno', 'Treviso', 'Venezia', 'Padova', 'Rovigo',
       'Pordenone', 'Udine', 'Gorizia', 'Trieste', 'Piacenza', 'Parma',
       "Reggio nell'Emilia", 'Modena', 'Bologna', 'Ferrara', 'Ravenna',
       'Forlì-Cesena', 'Rimini', 'Massa-Carrara', 'Lucca', 'Pistoia',
       'Firenze', 'Prato', 'Livorno', 'Pisa', 'Arezzo', 'Siena',
       'Grosseto', 'Perugia', 'Terni', 'Pesaro e Urbino', 'Ancona',
       'Macerata', 'Ascoli Piceno', 'Viterbo', 'Rieti', 'Roma', 'Latina',
       'Frosinone', "L'Aquila", 'Teramo', 'Pescara', 'Chieti', 'Isernia',
       'Campobasso', 'Caserta', 'Benevento', 'Avellino', 'Salerno',
       'Foggi

In [14]:
cereals1 =  cereals.loc[ cereals['Data_type'] == 'total production - quintals ']

In [15]:
cereals1['total_production'] = cereals1['Value'][cereals1['Data_type']=='total production - quintals ']
cereals1.head(10)   

,City,Data_type,Type_crop,Year,Value,total_production
17,Torino,total production - quintals,common wheat,2006,1197000.00,1197000.00
18,Torino,total production - quintals,common wheat,2007,1333100.00,1333100.00
19,Torino,total production - quintals,common wheat,2008,1216800.00,1216800.00
20,Torino,total production - quintals,common wheat,2009,1149083.00,1149083.00
21,Torino,total production - quintals,common wheat,2010,1092994.00,1092994.00
22,Torino,total production - quintals,common wheat,2011,1158380.00,1158380.00
23,Torino,total production - quintals,common wheat,2012,1352399.00,1352399.00
24,Torino,total production - quintals,common wheat,2013,1047489.00,1047489.00
25,Torino,total production - quintals,common wheat,2014,1037751.00,1037751.00
26,Torino,total production - quintals,common wheat,2015,1005000.00,1005000.00


In [16]:
cereals2 =  cereals.loc[ cereals['Data_type'] == 'total area - hectares']
cereals2 

,City,Data_type,Type_crop,Year,Value
0,Torino,total area - hectares,common wheat,2006,17100.00
1,Torino,total area - hectares,common wheat,2007,21502.00
2,Torino,total area - hectares,common wheat,2008,23400.00
3,Torino,total area - hectares,common wheat,2009,23167.00
4,Torino,total area - hectares,common wheat,2010,20904.00
...,...,...,...,...,...
66246,Ascoli Piceno,total area - hectares,common winter wheat and spelt,2021,5761.00
66248,Potenza,total area - hectares,common winter wheat and spelt,2021,5752.00
66249,Matera,total area - hectares,common winter wheat and spelt,2021,1200.00
66250,Fermo,total area - hectares,common winter wheat and spelt,2021,3239.00


In [17]:
cereals2['total_area_ha'] = cereals2['Value'][cereals2['Data_type']=='total area - hectares']
cereals2.tail(10)   

,City,Data_type,Type_crop,Year,Value,total_area_ha
66238,Verbano-Cusio-Ossola,total area - hectares,spring cereal mixtures (mixed grain other than...,2021,10.00,10.00
66239,Arezzo,total area - hectares,spring cereal mixtures (mixed grain other than...,2021,40.00,40.00
66240,Pesaro e Urbino,total area - hectares,common winter wheat and spelt,2021,755.00,755.00
66242,Ancona,total area - hectares,common winter wheat and spelt,2021,218.00,218.00
66244,Macerata,total area - hectares,common winter wheat and spelt,2021,3865.00,3865.00
66246,Ascoli Piceno,total area - hectares,common winter wheat and spelt,2021,5761.00,5761.00
66248,Potenza,total area - hectares,common winter wheat and spelt,2021,5752.00,5752.00
66249,Matera,total area - hectares,common winter wheat and spelt,2021,1200.00,1200.00
66250,Fermo,total area - hectares,common winter wheat and spelt,2021,3239.00,3239.00
66252,Bolzano / Bozen,total area - hectares,rice,2021,1.00,1.00


In [18]:
# Join both datasets 
cereals_total =  pd.merge_ordered(cereals1, cereals2, on=['Year', 'City', 'Type_crop'], how='left')
cereals_total

,City,Data_type_x,Type_crop,Year,Value_x,total_production,Data_type_y,Value_y,total_area_ha
0,Agrigento,total production - quintals,barley,2006,8970.00,8970.00,total area - hectares,345.00,345.00
1,Agrigento,total production - quintals,broad bean,2006,70490.00,70490.00,total area - hectares,3710.00,3710.00
2,Agrigento,total production - quintals,chick-peas,2006,1500.00,1500.00,total area - hectares,125.00,125.00
3,Agrigento,total production - quintals,dried kidney bean,2006,800.00,800.00,total area - hectares,50.00,50.00
4,Agrigento,total production - quintals,durum wheat,2006,1102000.00,1102000.00,total area - hectares,38000.00,38000.00
...,...,...,...,...,...,...,...,...,...
20426,Viterbo,total production - quintals,rye,2021,4790.00,4790.00,total area - hectares,190.00,190.00
20427,Viterbo,total production - quintals,rye and winter cereal mixtures (maslin),2021,4790.00,4790.00,total area - hectares,190.00,190.00
20428,Viterbo,total production - quintals,sorghum,2021,840.00,840.00,total area - hectares,25.00,25.00
20429,Viterbo,total production - quintals,sweet lupin,2021,260.00,260.00,total area - hectares,20.00,20.00


In [19]:
cereals_total =cereals_total.drop(columns =['Data_type_x','Data_type_y', 'Value_x', 'Value_y' ])
cereals_total

,City,Type_crop,Year,total_production,total_area_ha
0,Agrigento,barley,2006,8970.00,345.00
1,Agrigento,broad bean,2006,70490.00,3710.00
2,Agrigento,chick-peas,2006,1500.00,125.00
3,Agrigento,dried kidney bean,2006,800.00,50.00
4,Agrigento,durum wheat,2006,1102000.00,38000.00
...,...,...,...,...,...
20426,Viterbo,rye,2021,4790.00,190.00
20427,Viterbo,rye and winter cereal mixtures (maslin),2021,4790.00,190.00
20428,Viterbo,sorghum,2021,840.00,25.00
20429,Viterbo,sweet lupin,2021,260.00,20.00


In [21]:
# Transform values from quintals to tonnes
cereals_total['total_production'] = cereals_total['total_production']/10
cereals_total

,City,Type_crop,Year,total_production,total_area_ha
0,Agrigento,barley,2006,897.00,345.00
1,Agrigento,broad bean,2006,7049.00,3710.00
2,Agrigento,chick-peas,2006,150.00,125.00
3,Agrigento,dried kidney bean,2006,80.00,50.00
4,Agrigento,durum wheat,2006,110200.00,38000.00
...,...,...,...,...,...
20426,Viterbo,rye,2021,479.00,190.00
20427,Viterbo,rye and winter cereal mixtures (maslin),2021,479.00,190.00
20428,Viterbo,sorghum,2021,84.00,25.00
20429,Viterbo,sweet lupin,2021,26.00,20.00


In [26]:
# Import climate dataset
climate_mean = pd.read_csv('climate2006-2021_mean.csv')
climate_mean.head()

,Unnamed: 0.1,City,Year,Unnamed: 0,PS,TS,QV2M,RH2M,WS2M,GWETTOP,T2M_MAX,T2M_MIN,GWETPROF,GWETROOT,CLOUD_AMT,T2M_RANGE,PRECTOTCORR
0,0,Abano Terme,2006,6,100.76,13.30,7.32,71.53,1.56,0.66,25.65,3.84,0.65,0.67,56.54,21.80,1.68
1,1,Abano Terme,2007,19,100.66,14.06,7.23,68.75,1.59,0.61,26.09,4.63,0.61,0.63,54.11,21.47,1.94
2,2,Abano Terme,2008,32,100.63,13.64,7.72,73.95,1.67,0.69,24.69,4.30,0.67,0.70,59.45,20.38,2.77
3,3,Abano Terme,2009,45,100.47,13.71,7.72,72.75,1.69,0.70,24.92,4.11,0.70,0.72,57.77,20.82,2.45
4,4,Abano Terme,2010,58,100.33,12.55,7.54,75.60,1.70,0.75,23.83,2.73,0.73,0.76,61.55,21.10,3.15


In [27]:
# Check cities names
climate_mean.City.unique()

array(['Abano Terme', 'Abbadia Lariana', 'Abbadia San Salvatore', ...,
       'Zogno', 'Zola Predosa', 'Zoppola'], dtype=object)

In [28]:
# Select cities that correlate with crop data 

climate_mean = climate_mean.apply(lambda row: row[climate_mean['City'].isin(['Agrigento', 'Alessandria', 'Ancona', 'Arezzo', 'Ascoli Piceno',
                                                                             'Asti', 'Avellino', 'Bari', 'Barletta-Andria-Trani', 'Belluno',
                                                                             'Benevento', 'Bergamo', 'Biella', 'Bologna', 'Bolzano / Bozen',
                                                                             'Brescia', 'Brindisi', 'Cagliari', 'Caltanissetta', 'Campobasso',
                                                                             'Carbonia-Iglesias', 'Caserta', 'Catania', 'Catanzaro', 'Chieti',
                                                                             'Como', 'Cosenza', 'Cremona', 'Crotone', 'Cuneo', 'Enna', 'Fermo',
                                                                             'Ferrara', 'Firenze', 'Foggia', 'Forlì-Cesena', 'Frosinone',
                                                                             'Genova', 'Gorizia', 'Grosseto', 'Imperia', 'Isernia', "L'Aquila",
                                                                             'La Spezia', 'Latina', 'Lecce', 'Lecco', 'Livorno', 'Lodi',
                                                                             'Lucca', 'Macerata', 'Mantova', 'Massa-Carrara', 'Matera',
                                                                             'Medio Campidano', 'Messina', 'Milano', 'Modena',
                                                                             'Monza e della Brianza', 'Napoli', 'Novara', 'Nuoro', 'Ogliastra',
                                                                             'Olbia-Tempio', 'Oristano', 'Padova', 'Palermo', 'Parma', 'Pavia',
                                                                             'Perugia', 'Pesaro e Urbino', 'Pescara', 'Piacenza', 'Pisa',
                                                                             'Pistoia', 'Pordenone', 'Potenza', 'Prato', 'Ragusa', 'Ravenna',
                                                                             'Reggio di Calabria', "Reggio nell'Emilia", 'Rieti', 'Rimini',
                                                                             'Roma', 'Rovigo', 'Salerno', 'Sassari', 'Savona', 'Siena',
                                                                             'Siracusa', 'Sondrio', 'Sud Sardegna', 'Taranto', 'Teramo',
                                                                             'Terni', 'Torino', 'Trapani', 'Trentino Alto Adige / Südtirol',
                                                                             'Trento', 'Treviso', 'Trieste', 'Udine',
                                                                             "Valle d'Aosta / Vallée d'Aoste", 'Varese', 'Venezia',
                                                                             'Verbano-Cusio-Ossola', 'Vercelli', 'Verona', 'Vibo Valentia',
                                                                             'Vicenza', 'Viterbo'])])

climate_mean.head()

,Unnamed: 0.1,City,Year,Unnamed: 0,PS,TS,QV2M,RH2M,WS2M,GWETTOP,T2M_MAX,T2M_MIN,GWETPROF,GWETROOT,CLOUD_AMT,T2M_RANGE,PRECTOTCORR
603,603,Agrigento,2006,6,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58
604,604,Agrigento,2007,19,96.80,17.18,8.35,71.72,2.75,0.55,28.65,7.18,0.59,0.60,41.58,21.47,1.78
605,605,Agrigento,2008,32,96.83,16.91,8.27,71.72,2.66,0.58,28.02,6.84,0.61,0.61,39.13,21.19,1.61
606,606,Agrigento,2009,45,96.63,16.22,8.67,75.45,2.73,0.70,26.94,7.30,0.72,0.73,42.54,19.64,2.82
607,607,Agrigento,2010,58,96.57,16.20,8.60,75.71,2.87,0.69,26.97,6.97,0.71,0.72,46.03,19.99,2.79


In [29]:
climate_mean.City.unique()

array(['Agrigento', 'Alessandria', 'Ancona',
       "Valle d'Aosta / Vallée d'Aoste", 'Arezzo', 'Ascoli Piceno',
       'Asti', 'Avellino', 'Bari', 'Barletta-Andria-Trani', 'Belluno',
       'Benevento', 'Bergamo', 'Biella', 'Bologna', 'Bolzano / Bozen',
       'Brescia', 'Brindisi', 'Sud Sardegna', 'Caltanissetta',
       'Campobasso', 'Carbonia-Iglesias', 'Caserta', 'Catania',
       'Catanzaro', 'Chieti', 'Latina', 'Como', 'Cosenza', 'Cremona',
       'Crotone', 'Cuneo', 'Enna', 'Fermo', 'Ferrara', 'Firenze',
       'Foggia', 'Frosinone', 'Genova', 'Gorizia', 'Grosseto', 'Imperia',
       'Isernia', 'La Spezia', 'Ogliastra', 'Lecce', 'Lecco', 'Livorno',
       'Lodi', 'Lucca', 'Macerata', 'Mantova', 'Matera', 'Messina',
       'Milano', 'Modena', 'Treviso', 'Monza e della Brianza', 'Napoli',
       'Novara', 'Nuoro', 'Olbia-Tempio', 'Oristano', 'Padova', 'Palermo',
       'Parma', 'Pavia', 'Perugia', 'Pesaro e Urbino', 'Pescara',
       'Piacenza', 'Pisa', 'Pistoia', 'Pordenone', 'P

In [31]:
# Join both crop and climate datasets 
climate_cereal = pd.merge_ordered(cereals_total, climate_mean,  left_by="City").fillna(0)

In [63]:
climate_cereal.head()

,City,Type_crop,Year,Production_tonnes,Unnamed: 0.1,Unnamed: 0,PS,TS,QV2M,RH2M,WS2M,GWETTOP,T2M_MAX,T2M_MIN,GWETPROF,GWETROOT,CLOUD_AMT,T2M_RANGE,PRECTOTCORR
0,Torino,common wheat,2006,119700.00,37930.00,6.00,93.51,10.72,6.16,64.90,1.33,0.62,24.23,1.00,0.61,0.61,58.07,23.23,1.79
1,Torino,rye,2006,494.00,37930.00,6.00,93.51,10.72,6.16,64.90,1.33,0.62,24.23,1.00,0.61,0.61,58.07,23.23,1.79
2,Torino,barley,2006,28080.00,37930.00,6.00,93.51,10.72,6.16,64.90,1.33,0.62,24.23,1.00,0.61,0.61,58.07,23.23,1.79
3,Torino,oats,2006,874.00,37930.00,6.00,93.51,10.72,6.16,64.90,1.33,0.62,24.23,1.00,0.61,0.61,58.07,23.23,1.79
4,Torino,durum wheat,2006,1755.00,37930.00,6.00,93.51,10.72,6.16,64.90,1.33,0.62,24.23,1.00,0.61,0.61,58.07,23.23,1.79


In [32]:
climate_cereals  =  climate_cereal.to_csv('cereals_climate_2006-2021.csv', index=False)

In [33]:
cereals = pd.read_csv('cereals_climate_2006-2021.csv',skipinitialspace=True)
cereals.head()

,City,Type_crop,Year,production_tonnes,total_area_ha,Unnamed: 0.1,Unnamed: 0,PS,TS,QV2M,RH2M,WS2M,GWETTOP,T2M_MAX,T2M_MIN,GWETPROF,GWETROOT,CLOUD_AMT,T2M_RANGE,PRECTOTCORR
0,Agrigento,barley,2006,897.00,345.00,603.00,6.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58
1,Agrigento,broad bean,2006,7049.00,3710.00,603.00,6.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58
2,Agrigento,chick-peas,2006,150.00,125.00,603.00,6.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58
3,Agrigento,dried kidney bean,2006,80.00,50.00,603.00,6.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58
4,Agrigento,durum wheat,2006,110200.00,38000.00,603.00,6.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58


Import fertilizers data 

In [34]:
fertilizer = pd.read_csv('Italy_crop_data/fertilizer2006-2021.csv',skipinitialspace=True)
fertilizer.head()

,City,Type_fertilizer,Year,Fertilizers_tonnes
0,Torino,calcium cyanamide,2006,122.70
1,Torino,calcium cyanamide,2007,181.20
2,Torino,calcium cyanamide,2008,522.40
3,Torino,calcium cyanamide,2009,205.10
4,Torino,calcium cyanamide,2010,5.40


Join crop and fertilizers datasets 

In [40]:
# Join both datasets 
crop_fertilizer_climate = pd.merge(climate_cereal , fertilizer, on=['Year', 'City'], how='left').fillna(0)
crop_fertilizer_climate

,City,Type_crop,Year,production_tonnes,total_area_ha,Unnamed: 0.1,Unnamed: 0,PS,TS,QV2M,...,GWETTOP,T2M_MAX,T2M_MIN,GWETPROF,GWETROOT,CLOUD_AMT,T2M_RANGE,PRECTOTCORR,Type_fertilizer,Fertilizers_tonnes
0,Agrigento,barley,2006,897.00,345.00,603.00,6.00,96.87,16.70,8.33,...,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,calcium cyanamide,0.00
1,Agrigento,barley,2006,897.00,345.00,603.00,6.00,96.87,16.70,8.33,...,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,nitrogen-potassium,27.70
2,Agrigento,barley,2006,897.00,345.00,603.00,6.00,96.87,16.70,8.33,...,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,peaty-amend,174.50
3,Agrigento,barley,2006,897.00,345.00,603.00,6.00,96.87,16.70,8.33,...,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,organic-nitrogen,760.50
4,Agrigento,barley,2006,897.00,345.00,603.00,6.00,96.87,16.70,8.33,...,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,organic,1343.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206846,Sud Sardegna,winter barley,2021,13076.70,5428.00,7170.00,136.00,100.92,19.17,10.55,...,0.89,23.78,13.52,0.56,0.58,49.54,10.26,2.78,nitrogen-potassium,113.00
206847,Sud Sardegna,winter barley,2021,13076.70,5428.00,7170.00,136.00,100.92,19.17,10.55,...,0.89,23.78,13.52,0.56,0.58,49.54,10.26,2.78,organic-nitrogen,221.00
206848,Sud Sardegna,winter barley,2021,13076.70,5428.00,7170.00,136.00,100.92,19.17,10.55,...,0.89,23.78,13.52,0.56,0.58,49.54,10.26,2.78,organic,0.00
206849,Sud Sardegna,winter barley,2021,13076.70,5428.00,7170.00,136.00,100.92,19.17,10.55,...,0.89,23.78,13.52,0.56,0.58,49.54,10.26,2.78,peaty-amend,0.00


In [41]:
# Drop Columns
crop_fertilizer_climate = crop_fertilizer_climate.drop(columns =[ 'Unnamed: 0.1','Unnamed: 0'])
crop_fertilizer_climate

,City,Type_crop,Year,production_tonnes,total_area_ha,PS,TS,QV2M,RH2M,WS2M,GWETTOP,T2M_MAX,T2M_MIN,GWETPROF,GWETROOT,CLOUD_AMT,T2M_RANGE,PRECTOTCORR,Type_fertilizer,Fertilizers_tonnes
0,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,calcium cyanamide,0.00
1,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,nitrogen-potassium,27.70
2,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,peaty-amend,174.50
3,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,organic-nitrogen,760.50
4,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,organic,1343.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206846,Sud Sardegna,winter barley,2021,13076.70,5428.00,100.92,19.17,10.55,76.47,4.35,0.89,23.78,13.52,0.56,0.58,49.54,10.26,2.78,nitrogen-potassium,113.00
206847,Sud Sardegna,winter barley,2021,13076.70,5428.00,100.92,19.17,10.55,76.47,4.35,0.89,23.78,13.52,0.56,0.58,49.54,10.26,2.78,organic-nitrogen,221.00
206848,Sud Sardegna,winter barley,2021,13076.70,5428.00,100.92,19.17,10.55,76.47,4.35,0.89,23.78,13.52,0.56,0.58,49.54,10.26,2.78,organic,0.00
206849,Sud Sardegna,winter barley,2021,13076.70,5428.00,100.92,19.17,10.55,76.47,4.35,0.89,23.78,13.52,0.56,0.58,49.54,10.26,2.78,peaty-amend,0.00


In [42]:
crop_fertilizer_climate  =  crop_fertilizer_climate.to_csv('cereal_fertilizer_climate_2006-2021.csv', index=False)

In [43]:
crop_fertilizer_climate= pd.read_csv('cereal_fertilizer_climate_2006-2021.csv',skipinitialspace=True)
crop_fertilizer_climate.head()

,City,Type_crop,Year,production_tonnes,total_area_ha,PS,TS,QV2M,RH2M,WS2M,GWETTOP,T2M_MAX,T2M_MIN,GWETPROF,GWETROOT,CLOUD_AMT,T2M_RANGE,PRECTOTCORR,Type_fertilizer,Fertilizers_tonnes
0,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,calcium cyanamide,0.00
1,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,nitrogen-potassium,27.70
2,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,peaty-amend,174.50
3,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,organic-nitrogen,760.50
4,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,0.58,28.51,6.77,0.61,0.62,39.63,21.74,1.58,organic,1343.70


Join locations long and lat to dataset


In [44]:
geo = pd.read_csv('it_locations2.csv',skipinitialspace=True)
geo.head()

,City,lat,lon
0,Roma,41.89,12.48
1,Milano,45.47,9.19
2,Napoli,40.83,14.25
3,Turin,45.07,7.70
4,Palermo,38.12,13.36


In [45]:
# Select cities that correlate with crop data 

geo = geo.apply(lambda row: row[geo['City'].isin(['Agrigento', 'Alessandria', 'Ancona', 'Arezzo', 'Ascoli Piceno',
                                                                             'Asti', 'Avellino', 'Bari', 'Barletta-Andria-Trani', 'Belluno',
                                                                             'Benevento', 'Bergamo', 'Biella', 'Bologna', 'Bolzano / Bozen',
                                                                             'Brescia', 'Brindisi', 'Cagliari', 'Caltanissetta', 'Campobasso',
                                                                             'Carbonia-Iglesias', 'Caserta', 'Catania', 'Catanzaro', 'Chieti',
                                                                             'Como', 'Cosenza', 'Cremona', 'Crotone', 'Cuneo', 'Enna', 'Fermo',
                                                                             'Ferrara', 'Firenze', 'Foggia', 'Forlì-Cesena', 'Frosinone',
                                                                             'Genova', 'Gorizia', 'Grosseto', 'Imperia', 'Isernia', "L'Aquila",
                                                                             'La Spezia', 'Latina', 'Lecce', 'Lecco', 'Livorno', 'Lodi',
                                                                             'Lucca', 'Macerata', 'Mantova', 'Massa-Carrara', 'Matera',
                                                                             'Medio Campidano', 'Messina', 'Milano', 'Modena',
                                                                             'Monza e della Brianza', 'Napoli', 'Novara', 'Nuoro', 'Ogliastra',
                                                                             'Olbia-Tempio', 'Oristano', 'Padova', 'Palermo', 'Parma', 'Pavia',
                                                                             'Perugia', 'Pesaro e Urbino', 'Pescara', 'Piacenza', 'Pisa',
                                                                             'Pistoia', 'Pordenone', 'Potenza', 'Prato', 'Ragusa', 'Ravenna',
                                                                             'Reggio di Calabria', "Reggio nell'Emilia", 'Rieti', 'Rimini',
                                                                             'Roma', 'Rovigo', 'Salerno', 'Sassari', 'Savona', 'Siena',
                                                                             'Siracusa', 'Sondrio', 'Sud Sardegna', 'Taranto', 'Teramo',
                                                                             'Terni', 'Torino', 'Trapani', 'Trentino Alto Adige / Südtirol',
                                                                             'Trento', 'Treviso', 'Trieste', 'Udine',
                                                                             "Valle d'Aosta / Vallée d'Aoste", 'Varese', 'Venezia',
                                                                             'Verbano-Cusio-Ossola', 'Vercelli', 'Verona', 'Vibo Valentia',
                                                                             'Vicenza', 'Viterbo'])])
geo.head()

,City,lat,lon
0,Roma,41.89,12.48
1,Milano,45.47,9.19
2,Napoli,40.83,14.25
4,Palermo,38.12,13.36
5,Genova,44.41,8.93


In [46]:
geo.City.unique()

array(['Roma', 'Milano', 'Napoli', 'Palermo', 'Genova', 'Bologna',
       'Firenze', 'Bari', 'Catania', 'Venezia', 'Verona', 'Messina',
       'Padova', 'Trieste', 'Taranto', 'Brescia', 'Parma', 'Prato',
       'Modena', 'Reggio di Calabria', "Reggio nell'Emilia", 'Perugia',
       'Ravenna', 'Livorno', 'Cagliari', 'Foggia', 'Rimini', 'Salerno',
       'Ferrara', 'Latina', 'Monza e della Brianza', 'Siracusa',
       'Bergamo', 'Pescara', 'Trento', 'Sassari', 'Vicenza', 'Terni',
       'Bolzano / Bozen', 'Novara', 'Piacenza', 'Ancona', 'Udine',
       'Arezzo', 'Forlì-Cesena', 'Lecce', 'Pesaro e Urbino',
       'Barletta-Andria-Trani', 'Alessandria', 'La Spezia', 'Pistoia',
       'Pisa', 'Catanzaro', 'Lucca', 'Brindisi', 'Treviso', 'Como',
       'Grosseto', 'Varese', 'Asti', 'Caserta', 'Ragusa', 'Pavia',
       'Cremona', 'Trapani', 'Viterbo', 'Cosenza', 'Potenza', 'Crotone',
       'Massa-Carrara', 'Caltanissetta', 'Benevento', 'Savona', 'Matera',
       'Olbia-Tempio', 'Agrigento', 

In [47]:
# Join both crop and locations datasets 
final_cereal_crop = pd.merge_ordered(crop_fertilizer_climate, geo,  left_by="City").fillna(0)

In [48]:
final_cereal_crop

,City,Type_crop,Year,production_tonnes,total_area_ha,PS,TS,QV2M,RH2M,WS2M,...,T2M_MIN,GWETPROF,GWETROOT,CLOUD_AMT,T2M_RANGE,PRECTOTCORR,Type_fertilizer,Fertilizers_tonnes,lat,lon
0,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,...,6.77,0.61,0.62,39.63,21.74,1.58,calcium cyanamide,0.00,37.31,13.58
1,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,...,6.77,0.61,0.62,39.63,21.74,1.58,nitrogen-potassium,27.70,37.31,13.58
2,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,...,6.77,0.61,0.62,39.63,21.74,1.58,peaty-amend,174.50,37.31,13.58
3,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,...,6.77,0.61,0.62,39.63,21.74,1.58,organic-nitrogen,760.50,37.31,13.58
4,Agrigento,barley,2006,897.00,345.00,96.87,16.70,8.33,71.95,2.61,...,6.77,0.61,0.62,39.63,21.74,1.58,organic,1343.70,37.31,13.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206846,Sud Sardegna,winter barley,2021,13076.70,5428.00,100.92,19.17,10.55,76.47,4.35,...,13.52,0.56,0.58,49.54,10.26,2.78,nitrogen-potassium,113.00,0.00,0.00
206847,Sud Sardegna,winter barley,2021,13076.70,5428.00,100.92,19.17,10.55,76.47,4.35,...,13.52,0.56,0.58,49.54,10.26,2.78,organic-nitrogen,221.00,0.00,0.00
206848,Sud Sardegna,winter barley,2021,13076.70,5428.00,100.92,19.17,10.55,76.47,4.35,...,13.52,0.56,0.58,49.54,10.26,2.78,organic,0.00,0.00,0.00
206849,Sud Sardegna,winter barley,2021,13076.70,5428.00,100.92,19.17,10.55,76.47,4.35,...,13.52,0.56,0.58,49.54,10.26,2.78,peaty-amend,0.00,0.00,0.00


In [49]:
# save final dataset
final_cereal_crop = final_cereal_crop.to_csv('cereal_final_dataset_2006-2021.csv',index=False )